In [7]:
from splinter import Browser
from bs4 import BeautifulSoup
import re 
import pandas as pd
from splinter.exceptions import ElementDoesNotExist
import requests

In [8]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [9]:
def browser_init(url):
    executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
    browser = Browser('chrome', **executable_path, headless=False)
    browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    browser.quit() 
    return soup

In [10]:
url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
soup = browser_init(url)
title_div = soup.find('div', class_='content_title').text
p_div = soup.find('div' , class_ = 'article_teaser_body').text.strip()
print(f'{title_div}\n{p_div}')

Robotic Toolkit Added to NASA's Mars 2020 Rover
The bit carousel, which lies at the heart of the rover's Sample Caching System, is now aboard NASA's newest rover.


# PL Mars Space Images - Featured Image

In [11]:
base_url = "https://www.jpl.nasa.gov"
soup = browser_init(base_url + "/spaceimages/?search=&category=Mars")

In [12]:
featured_image = soup.find(class_ = 'carousel_item')
featured_image_a = featured_image.find('a')
featured_image_link =featured_image_a.get('data-fancybox-href') 
featured_image_link = base_url+featured_image_link
print(f'Link for the featured image(Mars) \n{featured_image_link}')

Link for the featured image(Mars) 
https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA16715_ip.jpg


# Mars Weather

In [13]:
twitter_base_url = 'https://twitter.com/marswxreport?lang=en'
soup = browser_init(twitter_base_url)
mars_weather_tweets = soup.find_all('p',class_ = 'TweetTextSize TweetTextSize--normal js-tweet-text tweet-text')
for tweet in mars_weather_tweets:
    weather_tweet = tweet.text
    if(weather_tweet.startswith('InSight sol')):
        mars_weather = weather_tweet
        break
print(mars_weather)

InSight sol 253 (2019-08-13) low -100.0ºC (-148.1ºF) high -25.5ºC (-13.9ºF)
winds from the SSE at 4.6 m/s (10.3 mph) gusting to 16.4 m/s (36.6 mph)
pressure at 7.60 hPapic.twitter.com/OnwaHAaYRH


# Mars Facts

In [16]:
mars_facts_url = 'https://space-facts.com/mars/'
tables = pd.read_html(mars_facts_url)
planet_facts = tables[1]

planet_facts
planet_facts

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [15]:
#converting to HTML 
facts_html_string = planet_facts.to_html(justify="left",index=False,header=False)
facts_html_string

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: left;">\n      <th>Mars - Earth Comparison</th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>Diameter:</td>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <td>Distance from Sun:</td>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <td>Length of Year:</td>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <td>Temperature:</td>\n      <td>-153 to 20 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>\n</table>'

# Mars Hemispheres

In [118]:
def get_image_info(soup):
    # Get image URL
    image_info = soup.find(class_="downloads")
    image_info_link = image_info.find("li").find("a")
    img_url = image_info_link.get("href")

    # Get hemisphere title
    title = soup.find(class_="content").find(class_="title").text
    
    # Create dict object
    mars_hemi_dict = {}
    mars_hemi_dict["title"] = title
    mars_hemi_dict["img_url"] = img_url
    
    return mars_hemi_dict

In [119]:
base_url = 'https://astrogeology.usgs.gov'
mars_hemi_url = base_url + '/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
soup = browser_init(mars_hemi_url)
links = []
parents = soup.select("#results-accordian")    
for parent in parents :
    
    for x in range(0,8, 2):
        a = parent.select('a',class_ = 'itemLink product-item')[x]
        links.append(base_url + a.get("href"))


In [120]:
mars_hemi_info = []
for link in links:
    soup = browser_init(link)
    mars_hemi_info.append(get_image_info(soup))
    
print(mars_hemi_info)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]
